<a href="https://colab.research.google.com/github/kjcoursera/TensorFlow/blob/main/IMDB_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [26]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info = True, as_supervised = True)

In [10]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

for s, l in train_data:
  training_sentences.append(s.numpy().decode('utf8'))
  training_labels.append(l.numpy())

for s, l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())

training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [30]:

vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer =  Tokenizer(num_words =  vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index =  tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen = max_length, truncating=trunc_type)

testing_sequences =  tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

In [32]:
reverse_word_index =  dict([(value,key) for (key, value) in word_index.items()])

def decode_review(text):
  return ' '.join([reverse_word_index.get(i,'?') for i in text])

print(decode_review(padded[10]))
print(training_sentences[10])


during a <OOV> night i was switching through the channels found this embarrassment of a movie what were they thinking br br if this is life after remote control for <OOV> <OOV> <OOV> no wonder she's gone nowhere br br and why did david keith take this role it's pathetic br br anyway i turned on the movie near the end so i didn't get much of the plot but this must've been the best part this nerdy college kid brings home this <OOV> ish girl this scene is straight out of the comic books or the cheap porn movies she calls the mother <OOV> <OOV> and kisses the father oh i didn't expect tongue great lines br br after
During a sleepless night, I was switching through the channels & found this embarrassment of a movie. What were they thinking?<br /><br />If this is life after "Remote Control" for Kari (Wuhrer) Salin, no wonder she's gone nowhere.<br /><br />And why did David Keith take this role? It's pathetic!<br /><br />Anyway, I turned on the movie near the end, so I didn't get much of the 

In [33]:
model =  tf.keras.Sequential([
                              tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
                              tf.keras.layers.Flatten(),
                              tf.keras.layers.Dense(6,activation='relu'),
                              tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss = 'binary_crossentropu', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_2 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [34]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_3 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [35]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs = num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 0.5068 - accuracy: 0.7244 - val_loss: 0.3675 - val_accuracy: 0.8342
Epoch 2/10
782/782 [==============================] - 3s 4ms/step - loss: 0.2387 - accuracy: 0.9065 - val_loss: 0.3654 - val_accuracy: 0.8423
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0857 - accuracy: 0.9789 - val_loss: 0.4488 - val_accuracy: 0.8296
Epoch 4/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0196 - accuracy: 0.9980 - val_loss: 0.5315 - val_accuracy: 0.8304
Epoch 5/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0055 - accuracy: 0.9996 - val_loss: 0.5859 - val_accuracy: 0.8289
Epoch 6/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.6317 - val_accuracy: 0.8311
Epoch 7/10
782/782 [==============================] - 4s 4ms/step - loss: 8.1683e-04 - accuracy: 1.0000 - val_loss: 0.6716 - val_accuracy: 0.8311
Ep

In [18]:
e = model.layers[0]
weights =  e.get_weights()[0]
print(weights.shape)

(10000, 16)


In [36]:
import io
out_v = io.open('vecs.tsv','w', encoding='utf-8')
out_m = io.open('meta.tsv','w',encoding='utf-8')
for word_num in range(1, vocab_size):
  word =  reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [38]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [39]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 64, 102, 12, 7, 478, 1200]]
